##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Clasificación de flores con aprendizaje por transferencia


<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/hub/tutorials/image_feature_vector"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/hub/tutorials/image_feature_vector.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/hub/tutorials/image_feature_vector.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/hub/tutorials/image_feature_vector.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a>
</td>
  <td>     <a href="https://tfhub.dev/google/imagenet/mobilenet_v2_035_128/feature_vector/2"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">Ver modelo en TF Hub</a>
</td>
</table>

¿Alguna vez ha visto una flor hermosa y se ha preguntado qué flor es? Bueno, no es la primera persona a quien le sucede esto. Entonces, ¡creemos una forma de identificar el tipo de flor a partir de una foto!

Para clasificar imágenes, se ha demostrado que hay un tipo particular de *red neuronal profunda*, llamada *red neuronal convolucional* que resulta particularmente potente para este caso. Sin embargo, las redes neuronales convolucionales modernas tienen millones de parámetros. Para entrenarlas desde cero hace falta contar con muchos datos de entrenamiento etiquetados y una gran potencia de cálculo (cientos de horas de GPU o más). Solamente tenemos tres mil fotos etiquetadas y queremos dedicarles mucho menos tiempo, por lo tanto, deberemos ser más inteligentes.

Usaremos una técnica llamada *aprendizaje por transferencia* en la que se toma una red previamente entrenada (entrenada con alrededor de un millón de imágenes), la usamos para extraer las funciones y entrenamos la capa nueva encima de nuestra propia tarea de clasificación de imágenes de flores.


## Preparación


In [ ]:
import collections
import io
import math
import os
import random
from six.moves import urllib

from IPython.display import clear_output, Image, display, HTML

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import tensorflow_hub as hub

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as sk_metrics
import time

## Conjunto de datos de flores

El conjunto de datos de flores está compuesto por imágenes de flores con 5 clases de etiquetas posibles.

Cuando entrenamos un modelo de aprendizaje automático, separamos los datos en conjuntos de datos de entrenamiento y prueba. Entrenaremos el modelo con nuestros datos de entrenamiento y luego evaluaremos en qué medida el modelo se desempeña correctamente con respecto a datos que no ha visto nunca (el conjunto de prueba).

Descarguemos los ejemplos de entrenamiento y prueba (puede demorar un poco) y separémoslos en conjuntos de prueba y entrenamiento.

Ejecutemos las siguientes dos celdas:

In [ ]:
FLOWERS_DIR = './flower_photos'
TRAIN_FRACTION = 0.8
RANDOM_SEED = 2018


def download_images():
  """If the images aren't already downloaded, save them to FLOWERS_DIR."""
  if not os.path.exists(FLOWERS_DIR):
    DOWNLOAD_URL = 'http://download.tensorflow.org/example_images/flower_photos.tgz'
    print('Downloading flower images from %s...' % DOWNLOAD_URL)
    urllib.request.urlretrieve(DOWNLOAD_URL, 'flower_photos.tgz')
    !tar xfz flower_photos.tgz
  print('Flower photos are located in %s' % FLOWERS_DIR)


def make_train_and_test_sets():
  """Split the data into train and test sets and get the label classes."""
  train_examples, test_examples = [], []
  shuffler = random.Random(RANDOM_SEED)
  is_root = True
  for (dirname, subdirs, filenames) in tf.gfile.Walk(FLOWERS_DIR):
    # The root directory gives us the classes
    if is_root:
      subdirs = sorted(subdirs)
      classes = collections.OrderedDict(enumerate(subdirs))
      label_to_class = dict([(x, i) for i, x in enumerate(subdirs)])
      is_root = False
    # The sub directories give us the image files for training.
    else:
      filenames.sort()
      shuffler.shuffle(filenames)
      full_filenames = [os.path.join(dirname, f) for f in filenames]
      label = dirname.split('/')[-1]
      label_class = label_to_class[label]
      # An example is the image file and it's label class.
      examples = list(zip(full_filenames, [label_class] * len(filenames)))
      num_train = int(len(filenames) * TRAIN_FRACTION)
      train_examples.extend(examples[:num_train])
      test_examples.extend(examples[num_train:])

  shuffler.shuffle(train_examples)
  shuffler.shuffle(test_examples)
  return train_examples, test_examples, classes


In [ ]:
# Download the images and split the images into train and test sets.
download_images()
TRAIN_EXAMPLES, TEST_EXAMPLES, CLASSES = make_train_and_test_sets()
NUM_CLASSES = len(CLASSES)

print('\nThe dataset has %d label classes: %s' % (NUM_CLASSES, CLASSES.values()))
print('There are %d training images' % len(TRAIN_EXAMPLES))
print('there are %d test images' % len(TEST_EXAMPLES))

## Exploración de los datos

El conjunto de datos de flores está compuesto por ejemplos que son imágenes etiquetadas de flores. Cada ejemplo contiene una imagen de una flor en JPEG con su etiqueta de clasificación: qué tipo de flor es. Veamos algunas imágenes junto con sus etiquetas.

In [ ]:
#@title Show some labeled images
def get_label(example):
  """Get the label (number) for given example."""
  return example[1]

def get_class(example):
  """Get the class (string) of given example."""
  return CLASSES[get_label(example)]

def get_encoded_image(example):
  """Get the image data (encoded jpg) of given example."""
  image_path = example[0]
  return tf.gfile.GFile(image_path, 'rb').read()

def get_image(example):
  """Get image as np.array of pixels for given example."""
  return plt.imread(io.BytesIO(get_encoded_image(example)), format='jpg')

def display_images(images_and_classes, cols=5):
  """Display given images and their labels in a grid."""
  rows = int(math.ceil(len(images_and_classes) / cols))
  fig = plt.figure()
  fig.set_size_inches(cols * 3, rows * 3)
  for i, (image, flower_class) in enumerate(images_and_classes):
    plt.subplot(rows, cols, i + 1)
    plt.axis('off')
    plt.imshow(image)
    plt.title(flower_class)

NUM_IMAGES = 15 #@param {type: 'integer'}
display_images([(get_image(example), get_class(example))
               for example in TRAIN_EXAMPLES[:NUM_IMAGES]])

## Construcción del modelo

Cargaremos un módulo de vectores de funciones de imágenes de [TF-Hub](https://tensorflow.org/hub), apilaremos un clasificador lineal arriba y agregaremos operaciones de entrenamiento y evaluación. En la siguiente celda se construye un grafo de TF que describe el modelo y su entrenamiento, pero no se ejecuta el entrenamiento (se hará en el paso siguiente).

In [ ]:
LEARNING_RATE = 0.01

tf.reset_default_graph()

# Load a pre-trained TF-Hub module for extracting features from images. We've
# chosen this particular module for speed, but many other choices are available.
image_module = hub.Module('https://tfhub.dev/google/imagenet/mobilenet_v2_035_128/feature_vector/2')

# Preprocessing images into tensors with size expected by the image module.
encoded_images = tf.placeholder(tf.string, shape=[None])
image_size = hub.get_expected_image_size(image_module)


def decode_and_resize_image(encoded):
  decoded = tf.image.decode_jpeg(encoded, channels=3)
  decoded = tf.image.convert_image_dtype(decoded, tf.float32)
  return tf.image.resize_images(decoded, image_size)


batch_images = tf.map_fn(decode_and_resize_image, encoded_images, dtype=tf.float32)

# The image module can be applied as a function to extract feature vectors for a
# batch of images.
features = image_module(batch_images)


def create_model(features):
  """Build a model for classification from extracted features."""
  # Currently, the model is just a single linear layer. You can try to add
  # another layer, but be careful... two linear layers (when activation=None)
  # are equivalent to a single linear layer. You can create a nonlinear layer
  # like this:
  # layer = tf.layers.dense(inputs=..., units=..., activation=tf.nn.relu)
  layer = tf.layers.dense(inputs=features, units=NUM_CLASSES, activation=None)
  return layer


# For each class (kind of flower), the model outputs some real number as a score
# how much the input resembles this class. This vector of numbers is often
# called the "logits".
logits = create_model(features)
labels = tf.placeholder(tf.float32, [None, NUM_CLASSES])

# Mathematically, a good way to measure how much the predicted probabilities
# diverge from the truth is the "cross-entropy" between the two probability
# distributions. For numerical stability, this is best done directly from the
# logits, not the probabilities extracted from them.
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels)
cross_entropy_mean = tf.reduce_mean(cross_entropy)

# Let's add an optimizer so we can train the network.
optimizer = tf.train.GradientDescentOptimizer(learning_rate=LEARNING_RATE)
train_op = optimizer.minimize(loss=cross_entropy_mean)

# The "softmax" function transforms the logits vector into a vector of
# probabilities: non-negative numbers that sum up to one, and the i-th number
# says how likely the input comes from class i.
probabilities = tf.nn.softmax(logits)

# We choose the highest one as the predicted class.
prediction = tf.argmax(probabilities, 1)
correct_prediction = tf.equal(prediction, tf.argmax(labels, 1))

# The accuracy will allow us to eval on our test set. 
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Entrenamiento de la red

Ahora que el modelo está creado, entrenémoslo y veamos cómo se desempeña con nuestro conjunto de prueba.

In [ ]:
# How long will we train the network (number of batches).
NUM_TRAIN_STEPS = 100 #@param {type: 'integer'}
# How many training examples we use in each step.
TRAIN_BATCH_SIZE = 10 #@param {type: 'integer'}
# How often to evaluate the model performance.
EVAL_EVERY = 10 #@param {type: 'integer'}

def get_batch(batch_size=None, test=False):
  """Get a random batch of examples."""
  examples = TEST_EXAMPLES if test else TRAIN_EXAMPLES
  batch_examples = random.sample(examples, batch_size) if batch_size else examples
  return batch_examples

def get_images_and_labels(batch_examples):
  images = [get_encoded_image(e) for e in batch_examples]
  one_hot_labels = [get_label_one_hot(e) for e in batch_examples]
  return images, one_hot_labels

def get_label_one_hot(example):
  """Get the one hot encoding vector for the example."""
  one_hot_vector = np.zeros(NUM_CLASSES)
  np.put(one_hot_vector, get_label(example), 1)
  return one_hot_vector

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(NUM_TRAIN_STEPS):
    # Get a random batch of training examples.
    train_batch = get_batch(batch_size=TRAIN_BATCH_SIZE)
    batch_images, batch_labels = get_images_and_labels(train_batch)
    # Run the train_op to train the model.
    train_loss, _, train_accuracy = sess.run(
        [cross_entropy_mean, train_op, accuracy],
        feed_dict={encoded_images: batch_images, labels: batch_labels})
    is_final_step = (i == (NUM_TRAIN_STEPS - 1))
    if i % EVAL_EVERY == 0 or is_final_step:
      # Get a batch of test examples.
      test_batch = get_batch(batch_size=None, test=True)
      batch_images, batch_labels = get_images_and_labels(test_batch)
      # Evaluate how well our model performs on the test set.
      test_loss, test_accuracy, test_prediction, correct_predicate = sess.run(
        [cross_entropy_mean, accuracy, prediction, correct_prediction],
        feed_dict={encoded_images: batch_images, labels: batch_labels})
      print('Test accuracy at step %s: %.2f%%' % (i, (test_accuracy * 100)))

In [ ]:
def show_confusion_matrix(test_labels, predictions):
  """Compute confusion matrix and normalize."""
  confusion = sk_metrics.confusion_matrix(
    np.argmax(test_labels, axis=1), predictions)
  confusion_normalized = confusion.astype("float") / confusion.sum(axis=1)
  axis_labels = list(CLASSES.values())
  ax = sns.heatmap(
      confusion_normalized, xticklabels=axis_labels, yticklabels=axis_labels,
      cmap='Blues', annot=True, fmt='.2f', square=True)
  plt.title("Confusion matrix")
  plt.ylabel("True label")
  plt.xlabel("Predicted label")

show_confusion_matrix(batch_labels, test_prediction)

## Predicciones incorrectas

Observemos un poco más de cerca los ejemplos de prueba en los que nuestro modelo se equivocó.

- ¿Hay algún ejemplo que tenga mal la etiqueta en nuestro conjunto de prueba?
- ¿Hay algún dato erróneo en el conjunto de prueba? ¿Imágenes que en realidad no son fotos de flores?
- ¿Hay imágenes que ayuden a entender por qué el modelo cometió el error?

In [ ]:
incorrect = [
    (example, CLASSES[prediction])
    for example, prediction, is_correct in zip(test_batch, test_prediction, correct_predicate)
    if not is_correct
]
display_images(
  [(get_image(example), "prediction: {0}\nlabel:{1}".format(incorrect_prediction, get_class(example)))
   for (example, incorrect_prediction) in incorrect[:20]])

## Ejercicios: Mejora del modelo

Hemos entrenado un modelo de base, ahora mejorémoslo para lograr una mayor exactitud. (Recuerde que cuando haga el cambio, deberá volver a ejecutar las celdas).

### Ejercicio 1:  Prueba con un modelo de imágenes diferentes.

Es muy fácil probar otros modelos de imágenes diferentes con TF-Hub. Simplemente, reemplace el `"https://tfhub.dev/google/imagenet/mobilenet_v2_050_128/feature_vector/2"` agregue la llamada a `hub.Module()` con un <em>handle</em> de un módulo diferente y vuelva a ejecutar todo el código. Puede ver todos los módulos de imágenes disponibles en [tfhub.dev](https://tfhub.dev/s?module-type=image-feature-vector).

Uno de los [módulos MobileNet V2](https://tfhub.dev/s?module-type=image-feature-vector&network-architecture=mobilenet-v2) sería una buena opción. Muchos módulos, incluidos los MobileNet, fueron entrenados con el [conjunto de datos ImageNet](https://www.tensorflow.org/datasets/catalog/imagenet2012) que tiene más de 1 millón de imágenes y 1000 clases. Al elegir una arquitectura de red se efectúa una compensación (<em>tradeoff</em>) entre la velocidad y la exactitud de la clasificación: los modelos como MobileNet o NASNet Mobile son arquitecturas tradicionales más rápidas y pequeñas, del mismo modo que Inception y ResNet fueron diseñados para la exactitud.

Para la arquitectura más amplia de Inception V3, también se pueden analizar los beneficios del entrenamiento previo en un dominio más cercano a su tarea: también se encuentra disponible como un [módulo entrenado con el conjunto de datos iNaturalist](https://tfhub.dev/google/inaturalist/inception_v3/feature_vector/1) sobre plantas y animales.

### Ejercicio 2: Agregado de una capa oculta.

Apile una capa oculta entre las características de imágenes extraídas y el clasificador lineal (de la función `create_model()` que figura arriba). Para crear una capa oculta no lineal con p. ej., 100 nodos, use [tf.layers.dense](https://www.tensorflow.org/api_docs/python/tf/compat/v1/layers/dense) con unidades en 100 y activación con `tf.nn.relu`. ¿El cambio de la capa oculta afecta a la precisión de la prueba? ¿Agregar una segunda capa oculta mejora la exactitud?

### Ejercicio 3: Cambio de los hiperparámetros.

¿El aumento de la *cantidad de pasos de entrenamiento* mejora la exactitud final? ¿Se puede *cambiar la tasa de aprendizaje* para que el modelo converja más rápido? ¿El *tamaño del lote* de entrenamiento afecta el desempeño del modelo?

### Ejercicio 4: Prueba con un optimizador diferente.

Reemplace el GradientDescentOptimizer básico por un optimizador más sofisticado; p. ej., con el [AdagradOptimizer](https://www.tensorflow.org/api_docs/python/tf/compat/v1/train/AdagradOptimizer). ¿Se nota alguna diferencia en el entrenamiento del modelo? Para más información acerca de los beneficios de los algoritmos de optimización, consulte [este artículo](http://ruder.io/optimizing-gradient-descent/).

## ¿Desea conocer más al respecto?

Si le interesa acceder a una versión más avanzada de este tutorial, consulte el [tutorial sobre reentrenamiento de imágenes con TensorFlow](https://www.tensorflow.org/hub/tutorials/image_retraining) en el que se muestra paso a paso como visualizar el entrenamiento con TensorBoard, además de compartir técnicas avanzadas como el aumento de conjuntos de datos mediante la distorsión de imágenes y el reemplazo de conjuntos de datos de flores para aprender un clasificador de imágenes en un conjunto de datos propio.

Puede obtener más información sobre TensorFlow en [tensorflow.org](http://tensorflow.org) y consultar la documentación de la API de TF-Hub, disponible en [tensorflow.org/hub](https://www.tensorflow.org/hub/). Hay módulos de TensorFlow disponibles en [tfhub.dev](http://tfhub.dev) que incluyen más módulos de vectores de características de imágenes y módulos de incorporación de textos.

Finalmente, consulte el [curso intensivo de aprendizaje automático](https://developers.google.com/machine-learning/crash-course/), un curso acelerado de Google que ofrece una introducción práctica al aprendizaje automático.